In [4]:
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt
import nibabel as nib
import nibabel.orientations as nio
import numpy as np

# Base dataset directory
base_dir = "/scratch/awias/data/Totalsegmentator_dataset_v201"

# Get all subject folders (s0000, s0001, ...)
subjects = sorted([d for d in os.listdir(base_dir) if d.startswith("s")])

for subj in subjects:
    ct_path = os.path.join(base_dir, subj, "ct.nii.gz")
    
    if not os.path.exists(ct_path):
        print(f"Skipping {subj}, no CT found.")
        continue

    try:
        # Load CT scan
        img_sitk = sitk.ReadImage(ct_path)
        arr = sitk.GetArrayFromImage(img_sitk)  # Shape: [z, y, x]
    except Exception as e:
        print(f"Error loading {subj}: {e}")
        img_nib = nib.load(ct_path)
        arr = img_nib.get_fdata(dtype=np.float32)
        
        zooms = img_nib.header.get_zooms() #Voxel sizes
        axs_code = nio.ornt2axcodes(nio.io_orientation(img_nib.affine)) #Image orientation
        data_shape_voxels = img_nib.header.get_data_shape() #Shape of data
        data_shape_mm = np.array(data_shape_voxels)*np.array(zooms) #Data measures
        data_type = img_nib.header.get_data_dtype() #Data type
        
        # Correct, if you use nib.save(img_nib, ...), you are saving the original image and metadata unchanged.
        # If you want to change metadata (e.g., header info, affine), you need to create a new Nifti1Image with your desired arr, affine, and header.

        # Example: change the header or affine, then save
        new_affine = img_nib.affine  # or modify as needed
        new_header = img_nib.header.copy()
        # You can modify new_header fields here, e.g.:
        # new_header['descrip'] = 'Modified CT scan'

        new_img = nib.Nifti1Image(arr, affine=new_affine, header=new_header)
        nib.save(new_img, os.path.join(base_dir, subj, "ct_modified.nii.gz"))

    break
# # Save as nii file
# # Save with original metadata if using nibabel
# if isinstance(img, nib.Nifti1Image):
#     new_img = nib.Nifti1Image(arr, affine=img.affine, header=img.header)
# else:
#     # For SimpleITK, convert to nibabel and use identity affine (or extract from sitk if needed)
#     new_img = nib.Nifti1Image(arr, affine=np.eye(4))
# nib.save(new_img, os.path.join(base_dir, subj, "ct_processed.nii.gz"))

Error loading s0000: Exception thrown in SimpleITK ImageFileReader_Execute: /tmp/SimpleITK-build/ITK/Modules/IO/NIFTI/src/itkNiftiImageIO.cxx:2135:
ITK ERROR: ITK only supports orthonormal direction cosines.  No orthonormal definition found!


In [2]:
new_header

In [9]:
zooms = img_nib.header.get_zooms() #Voxel sizes
axs_code = nio.ornt2axcodes(nio.io_orientation(img_nib.affine)) #Image orientation
data_shape_voxels = img_nib.header.get_data_shape() #Shape of data
data_shape_mm = np.array(data_shape_voxels)*np.array(zooms) #Data measures
data_type = img_nib.header.get_data_dtype() #Data type

print("CT Scan Information:")
print(f" - Voxel Sizes (mm): {zooms}")
print(f" - Orientation (RAS): {axs_code}")
print(f" - Data Shape (voxels): {data_shape_voxels}")
print(f" - Data Shape (mm): {data_shape_mm}")
print(f" - Data Type: {data_type}")

CT Scan Information:
 - Voxel Sizes (mm): (np.float32(1.5), np.float32(1.5000001), np.float32(1.5))
 - Orientation (RAS): ('R', 'A', 'S')
 - Data Shape (voxels): (294, 192, 179)
 - Data Shape (mm): [441.         288.00002289 268.5       ]
 - Data Type: int16


In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt

# Base dataset directory
base_dir = "/scratch/awias/data/Totalsegmentator_dataset_v201"

# Get all subject folders (s0000, s0001, ...)
subjects = sorted([d for d in os.listdir(base_dir) if d.startswith("s")])

for subj in subjects:
    ct_path = os.path.join(base_dir, subj, "ct.nii.gz")
    
    if not os.path.exists(ct_path):
        print(f"Skipping {subj}, no CT found.")
        continue

    # Load CT scan with nibabel
    img = nib.load(ct_path)
    arr = img.get_fdata()  # numpy array, shape: (x, y, z)

    # Pick the middle slice in z-direction
    mid_slice = arr.shape[2] // 2
    slice_img = arr[:, :, mid_slice]

    # Plot
    plt.figure(figsize=(5,5))
    plt.imshow(slice_img.T, cmap="gray", origin="lower")
    plt.title(f"{subj} - middle slice")
    plt.axis("off")
    plt.show()
